In [1]:
from urllib.parse import urlencode, urlparse, parse_qsl
import matplotlib.pyplot as plt
import requests
import pandas as pd
import numpy as np

pd.set_option('display.max_colwidth', 30)
pd.options.display.float_format = '{:,.2f}'.format

# Census & gmaps API Keys
from config import census_key, g_key

# GoogleMap object with methods:
        # extract_lat_lng(location)
        # search(keyword, radius = 1000, location=None)
        # detail(place_id, fields=["name", "rating", "formatted_phone_number", "formatted_address"])
from my_func import GoogleMapClient

In [5]:
# Reading and Cleaning DataSet (Data Source: Census.org)
uscities_data = pd.read_excel('resources/uscities_over50k.xlsx', header=None) 
uscities_data.columns = uscities_data.iloc[1]
uscities_data.drop([0,1], axis=0, inplace=True)
uscities_data.set_index('Rank', inplace=True)
uscities_data = uscities_data.rename(columns={"Census": "Cens_2010",
                                      "Estimates Base": "Base_2010",
                                      2010.0: "2010", 2011.0: "2011",
                                      2012.0: "2012", 2013.0: "2013",
                                      2014.0: "2014", 2015.0: "2015",
                                      2016.0: "2016", 2017.0: "2017",
                                      2018.0: "2018", 2019.0: "2019"})
uscities_data.head()
for column in uscities_data.columns[3:]:
    uscities_data[column]= uscities_data[column].astype(float)

In [6]:
uscities_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 788 entries, 1 to 788
Data columns (total 13 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   City       788 non-null    object
 1   Cens_2010  788 non-null    object
 2   Base_2010  788 non-null    object
 3   2010       788 non-null    int64 
 4   2011       788 non-null    int64 
 5   2012       788 non-null    int64 
 6   2013       788 non-null    int64 
 7   2014       788 non-null    int64 
 8   2015       788 non-null    int64 
 9   2016       788 non-null    int64 
 10  2017       788 non-null    int64 
 11  2018       788 non-null    int64 
 12  2019       788 non-null    int64 
dtypes: int64(10), object(3)
memory usage: 86.2+ KB


In [8]:
for row in uscities_data.itertuples():
    client = GoogleMapClient(api_key = g_key, address_or_postal_code = f'{row.City}')
    try: 
        uscities_data.loc[row.Index, 'Lat'] = client.lat
        uscities_data.loc[row.Index, 'Lng'] = client.lng
    except:
        pass
    if row.Index % 50 == 0: 
        print('Processing records: found ', row.Index,' cities')
print('\nProcessing is over\n')

Processing records: found  50  cities, skipped 
Processing records: found  100  cities, skipped 
Processing records: found  150  cities, skipped 
Processing records: found  200  cities, skipped 
Processing records: found  250  cities, skipped 
Processing records: found  300  cities, skipped 
Processing records: found  350  cities, skipped 
Processing records: found  400  cities, skipped 
Processing records: found  450  cities, skipped 
Processing records: found  500  cities, skipped 
Processing records: found  550  cities, skipped 
Processing records: found  600  cities, skipped 
Processing records: found  650  cities, skipped 
Processing records: found  700  cities, skipped 
Processing records: found  750  cities, skipped 


In [9]:
uscities_data.head()

1,City,Cens_2010,Base_2010,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,Lat,Lng
Rank,,,,,,,,,,,,,,,
1,"New York city, New York",8175133,8175031,8190209,8272948,8346693,8396091,8433806,8463049,8469153,8437478,8390081,8336817,40.712775,-74.005973
2,"Los Angeles city, California",3792621,3793139,3795512,3820876,3851202,3881622,3909901,3938568,3963226,3975788,3977596,3979576,34.052234,-118.243685
3,"Chicago city, Illinois",2695598,2695652,2697477,2708114,2719141,2725731,2727066,2724344,2716723,2711069,2701423,2693976,41.878114,-87.629798
4,"Houston city, Texas",2099451,2095517,2100280,2126032,2161593,2199391,2241826,2286908,2309544,2316750,2318573,2320268,29.760427,-95.369803
5,"Phoenix city, Arizona",1445632,1446691,1449038,1469796,1499274,1526491,1555445,1583690,1612199,1633560,1654675,1680992,33.448377,-112.074037


In [10]:
uscities_data.to_csv("resources/uscities_data.csv", encoding="utf-8", index=False)